# Google Colab set-up

In [0]:
from google.colab import drive
drive.mount('./gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at ./gdrive


The patches extracted from the images are stored in model1bmi/train

In [0]:
train_path = './train/'
test_path = './test'

In [0]:
mkdir models

# Getting dataset ready

In [0]:
!wget https://rdm.inesctec.pt/dataset/604dfdfa-1d37-41c6-8db1-e82683b8335a/resource/df04ea95-36a7-49a8-9b70-605798460c35/download/breasthistology.zip

The dataset is from https://rdm.inesctec.pt/dataset/nis-2017-003. The above zip is password-protected. Request the password from therein and extract it.

In [0]:
mkdir train/Normal train/Benign train/Invasive train/InSitu

In [0]:
# Credits: ICIAR2018 by ImagingLabs
class PatchExtractor:
    def __init__(self, img, patch_size, stride):
        '''
        :param img: :py:class:`~PIL.Image.Image`
        :param patch_size: integer, size of the patch
        :param stride: integer, size of the stride
        '''
        self.img = img
        self.size = patch_size
        self.stride = stride

    def extract_patches(self):
        """
        extracts all patches from an image
        :returns: A list of :py:class:`~PIL.Image.Image` objects.
        """
        wp, hp = self.shape()
        return [self.extract_patch((w, h)) for h in range(hp) for w in range(wp)]

    def extract_patch(self, patch):
        """
        extracts a patch from an input image
        :param patch: a tuple
        :rtype: :py:class:`~PIL.Image.Image`
        :returns: An :py:class:`~PIL.Image.Image` object.
        """
        return self.img.crop((
            patch[0] * self.stride,  # left
            patch[1] * self.stride,  # up
            patch[0] * self.stride + self.size,  # right
            patch[1] * self.stride + self.size  # down
        ))

    def shape(self):
        wp = int((self.img.width - self.size) / self.stride + 1)
        hp = int((self.img.height - self.size) / self.stride + 1)
        return wp, hp

In [0]:
import glob
from PIL import Image

LABELS = ['Normal', 'Benign', 'InSitu', 'Invasive']
IMAGE_SIZE = (2048, 1536)
PATCH_SIZE = 512

# This is the folder you'll find after extracting the dataset.
train_folder = './Training_data'
labels = {name: LABELS[index] for index in range(len(LABELS)) for name in glob.glob(train_folder + '/' + LABELS[index] + '/*.tif')}

for key, value in labels.items():
  try:
    with Image.open(key) as img:
      # the patch-size and stride is according to the paper
      extractor = PatchExtractor(img=img, patch_size=PATCH_SIZE, stride=256)
      patches = extractor.extract_patches()
      count = 0
      for p in patches:
        count += 1
        # print('./train/' + value + '/' + str(count) + '_' + key.split('/')[-1])
        p.save('./train/' + value + '/' + str(count) + '_' + key.split('/')[-1])
  except Exception as error:
    print('error with', key, error)

This will prepare a dataset with patches of size 512x512. 35 from each image. 

In [109]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

aug = ImageDataGenerator(rotation_range=360, brightness_range=(0.5, 1.5), horizontal_flip=True, vertical_flip=True, validation_split=0.15)

train_generator = aug.flow_from_directory(
    train_path,
    target_size=(512, 512),
    batch_size=bs,
    subset='training') # set as training data

validation_generator = aug.flow_from_directory(
    train_path, # same directory as training data
    target_size=(512, 512),
    batch_size=bs,
    subset='validation') # set as validation data

# test_datagen = ImageDataGenerator(rescale=1./255)

# test_generator = test_datagen.flow_from_directory(
#         test_path,
#         target_size=(512, 512),
#         batch_size=bs,
#         class_mode='categorical', shuffle=False)

Found 7067 images belonging to 4 classes.
Found 1244 images belonging to 4 classes.
Found 1260 images belonging to 4 classes.


In [0]:
train_generator.class_indices

{'Benign': 0, 'InSitu': 1, 'Invasive': 2, 'Normal': 3}

# Defining & Training the Model

In [0]:
bs = 32

In [0]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

In [0]:
class PatchWiseNet:
  @staticmethod
  def build(width, height, depth, classes, channels=1):
    # initialize the model along with the input shape to be
    # "channels last" and the channels dimension itself
    model = Sequential()
    pad = 'same'
    inputShape = (height, width, depth)
    chanDim = -1

    # if we are using "channels first", update the input shape
    # and channels dimension
    if K.image_data_format() == "channels_first":
      inputShape = (depth, height, width)
      chanDim = 1

    #       block 1
    model.add(Conv2D(filters=16, kernel_size=(3, 3), strides=1, padding=pad, input_shape=inputShape))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation("relu"))

    model.add(Conv2D(filters=16, kernel_size=(3, 3), strides=1, padding=pad))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation("relu"))

    model.add(Conv2D(filters=16, kernel_size=(2, 2), strides=2))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation("relu"))

    #     block 2
    model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=1, padding=pad))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation("relu"))

    model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=1, padding=pad))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation("relu"))

    model.add(Conv2D(filters=32, kernel_size=(2, 2), strides=2))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation("relu"))

    #     block 3
    model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=1, padding=pad))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation("relu"))

    model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=1, padding=pad))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation("relu"))

    model.add(Conv2D(filters=64, kernel_size=(2, 2), strides=2))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation("relu"))

    #     block 4
    model.add(Conv2D(filters=128, kernel_size=(3, 3), strides=1, padding=pad))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation("relu"))

    model.add(Conv2D(filters=128, kernel_size=(3, 3), strides=1, padding=pad))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation("relu"))

    model.add(Conv2D(filters=128, kernel_size=(3, 3), strides=1, padding=pad))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation("relu"))

    #     block 5
    model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=1, padding=pad))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation("relu"))

    model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=1, padding=pad))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation("relu"))

    model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=1, padding=pad))
    model.add(BatchNormalization(axis=chanDim))
    model.add(Activation("relu"))

    model.add(Conv2D(filters=channels, kernel_size=(1, 1), strides=1))

    model.add(Flatten())
    model.add(Dense(classes))
    model.add(Activation("softmax"))

    return model
    

In [0]:
model = PatchWiseNet.build(width=512, height=512, depth=3, classes=4)

In [0]:
model.summary()

In [0]:
from keras import optimizers
from keras import callbacks
import math

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
sgd = optimizers.SGD(lr=0.001)

filepath="./models/weights-{epoch:02d}-{val_acc:.2f}.hdf5"

mcp = callbacks.ModelCheckpoint(filepath=filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [mcp]


In [0]:
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=["accuracy"])

In [0]:
train_step=train_generator.n//train_generator.batch_size
valid_step=validation_generator.n//validation_generator.batch_size

In [0]:
model.fit_generator(
    train_generator,
    steps_per_epoch=train_step,
    validation_data=validation_generator,
    validation_steps=valid_step,
    epochs=50,
    callbacks=callbacks_list,
    verbose=1)

In [0]:
# Uncomment to load previously trained model
# from keras.models import load_model
# model = load_model('./models/weights-04-0.35.hdf5')

In [0]:
import PIL
import numpy

In [108]:
I = numpy.asarray(PIL.Image.open('./test/Invasive/10_15.tif'))
model.predict(I.reshape(-1,512, 512, 3) , verbose=1).argmax(axis=1)

1/1 [==============================] - 0s 26ms/step


array([2])

In [100]:
train_generator.class_indices

{'Benign': 0, 'InSitu': 1, 'Invasive': 2, 'Normal': 3}

# Convert the  Keras model to TF.js Layers format

In [0]:
# !pip install tensorflowjs

In [0]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, tfjs_target_dir)



--- OR ---



In [0]:
tensorflowjs_converter --input_format keras \
                       ./models/weights-10-0.79.hdf5 \
                       path/to/tfjs_target_dir

In [0]:
# If the above runs into some import errors, use it with tensorflow v0.6.4 and keras v2.1.6